# Szemantikus Kernel az OpenBnB MCP szerver integrációjával

Ez a jegyzetfüzet bemutatja, hogyan lehet a Szemantikus Kernel-t használni az OpenBnB MCP szerverrel, hogy valódi Airbnb szállásokat keressünk az MCPStdioPlugin segítségével. Az LLM hozzáféréshez az Azure AI Foundry-t használja. A környezeti változók beállításához kövesse a [Beállítási leckét](/00-course-setup/README.md).


## Importálja a szükséges csomagokat


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Az MCP Plugin kapcsolat létrehozása

Kapcsolódni fogunk az [OpenBnB MCP szerverhez](https://github.com/openbnb-org/mcp-server-airbnb) az MCPStdioPlugin segítségével. Ez a szerver az Airbnb keresési funkciót biztosítja az @openbnb/mcp-server-airbnb csomagon keresztül.


## Az ügyfél létrehozása

Ebben a példában az Azure AI Foundry-t fogjuk használni az LLM eléréséhez. Győződjön meg róla, hogy a környezeti változók megfelelően vannak beállítva.


## Környezet Konfigurációja

Konfiguráld az Azure OpenAI beállításait. Győződj meg róla, hogy a következő környezeti változók be vannak állítva:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Az OpenBnB MCP integráció megértése

Ez a jegyzetfüzet csatlakozik a **valós OpenBnB MCP szerverhez**, amely tényleges Airbnb keresési funkciókat biztosít.

### Hogyan működik:

1. **MCPStdioPlugin**: Standard bemenet/kimenet kommunikációt használ az MCP szerverrel
2. **Valós NPM csomag**: Letölti és futtatja az `@openbnb/mcp-server-airbnb` csomagot az npx segítségével
3. **Élő adatok**: Valós Airbnb ingatlanadatokat ad vissza az API-jukból
4. **Funkciók felfedezése**: Az ügynök automatikusan felismeri az MCP szerver által elérhető funkciókat

### Elérhető funkciók:

Az OpenBnB MCP szerver általában a következőket biztosítja:
- **search_listings** - Airbnb ingatlanok keresése hely és kritériumok alapján
- **get_listing_details** - Részletes információk lekérése konkrét ingatlanokról
- **check_availability** - Elérhetőség ellenőrzése konkrét dátumokra
- **get_reviews** - Vélemények lekérése ingatlanokról
- **get_host_info** - Információk lekérése az ingatlanok házigazdáiról

### Előfeltételek:

- **Node.js** telepítve a rendszereden
- **Internetkapcsolat** az MCP szerver csomag letöltéséhez
- **NPX** elérhető (a Node.js része)

### A kapcsolat tesztelése:

Az MCP szervert manuálisan tesztelheted az alábbi parancs futtatásával:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ez letölti és elindítja az OpenBnB MCP szervert, amelyhez a Semantic Kernel csatlakozik, hogy valós Airbnb adatokat érjen el.


## Az ügynök futtatása az OpenBnB MCP szerverrel

Most futtatjuk az AI ügynököt, amely csatlakozik az OpenBnB MCP szerverhez, hogy valódi Airbnb szállásokat keressen Stockholm városában 2 felnőtt és 1 gyerek számára. Nyugodtan módosítsd a `user_inputs` listát a keresési feltételek megváltoztatásához.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Összefoglaló
Gratulálunk! Sikeresen létrehoztál egy AI ügynököt, amely a Model Context Protocol (MCP) segítségével valós szálláskeresést integrál:

## Felhasznált technológiák:
- Semantic Kernel - Intelligens ügynökök építéséhez Azure OpenAI-val
- Azure AI Foundry - LLM képességekhez és csevegésbefejezéshez
- MCP (Model Context Protocol) - Szabványosított eszközintegrációhoz
- OpenBnB MCP Server - Valós Airbnb keresési funkcióhoz
- Node.js/NPX - Külső MCP szerver futtatásához

## Amit megtanultál:
- MCP integráció: Semantic Kernel ügynökök csatlakoztatása külső MCP szerverekhez
- Valós idejű adathozzáférés: Valós Airbnb ingatlanok keresése élő API-kon keresztül
- Protokollkommunikáció: Stdio kommunikáció használata az ügynök és az MCP szerver között
- Funkciófelfedezés: MCP szerverek elérhető funkcióinak automatikus felfedezése
- Streaming válaszok: Funkcióhívások valós idejű rögzítése és naplózása
- HTML megjelenítés: Ügynökválaszok formázása stílusos táblázatokkal és interaktív kijelzőkkel

## Következő lépések:
- További MCP szerverek integrálása (időjárás, repülőjáratok, éttermek)
- Több ügynökből álló rendszer építése MCP és A2A protokollok kombinálásával
- Egyedi MCP szerverek létrehozása saját adatforrásaidhoz
- Tartós beszélgetési memória megvalósítása munkamenetek között
- Az ügynök telepítése Azure Functions-be MCP szerverek összehangolásával
- Felhasználói hitelesítés és foglalási funkciók hozzáadása



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
